In [4]:
import pandas as pd
import numpy as np
from collections import OrderedDict

from sklearn.model_selection import TimeSeriesSplit

from sklearn.linear_model import LinearRegression

In [2]:
df_all = pd.read_csv('../data/processed/0.3-feature-processing.csv', index_col=0)
df = df_all[df_all['year'] > 1980]

**Group by race (year,round)**

In [5]:
groups_year = df.groupby(['year','round']).groups
groups_year = OrderedDict(groups_year)

**Define train/test splits based on year and round of the race**

In [14]:
list(groups_year.keys()).index((2019,1))

655

In [11]:
sorted_groups = [value for (_, value) in sorted(groups_year.items())] 

In [14]:
len(sorted_groups)

715

In [15]:
cv = 'undefined'

In [7]:
df.columns

Index(['driverId', 'gridStart', 'positionOrder', 'year', 'round', 'circuitId',
       'ageAtRace', 'ageAtDebut', 'yearsExperience', 'racingAtHome',
       'driverStandingsPoints', 'driverStandingsPosition',
       'driverStandingsWins', 'lastRaceRank', 'constructorStandingsPoints',
       'constructorStandingsPosition', 'constructorStandingsWins',
       'previousRaceGridStart', 'previousRacePosition', 'racesWon',
       'racesRetired', 'racesFinished', 'polePositions',
       'racesWonByConstructor', 'racesRetiredByConstructor',
       'percentageOfBestQuali'],
      dtype='object')

In [8]:
one_hot_columns = ['driverId','circuitId']

In [3]:
df = df[['gridStart','year','round','positionOrder']]

In [4]:
df_train=df[(df['year'] <= 2021) & (df['round']<19)]
df_validation=df[(df['year'] == 2021) & (df['round']==19)]

In [5]:
df_train.head(3)

,gridStart,year,round,positionOrder
0,1,2008,1,1
1,9,2008,2,5
2,3,2008,3,13


In [6]:
reg = LinearRegression().fit(df_train.iloc[:,:-3], df_train.iloc[:,-1:])
reg.score(df_train.iloc[:,:-3], df_train.iloc[:,-1:])

0.028457020490752383

In [7]:
prediction = reg.predict(df_validation.iloc[:,:-3])
df_prediction = pd.DataFrame(data=prediction,columns=['predictedOrder'])
df_validation = df_validation.reset_index(drop=True)

In [8]:
df_compare = pd.concat([df_validation, df_prediction], axis=1)

In [9]:
df_compare.head(20)

,gridStart,year,round,positionOrder,predictedOrder
0,10,2021,19,1,12.690997
1,12,2021,19,9,13.056627
2,0,2021,19,12,10.862848
3,9,2021,19,11,12.508183
4,4,2021,19,4,11.594108
5,11,2021,19,19,12.873812
6,1,2021,19,3,11.045663
7,3,2021,19,6,11.411293
8,2,2021,19,2,11.228478
9,8,2021,19,8,12.325368
